# <font color='red'>Geoplatform - Generate Lists - Dependencies</font>
<b><u>Contents:</b></u><br>
1) List Web Map Dependencies<br>
2) List Dashboard Dependencies<br>
2) List Experience Builder Dependencies<br>
<font color='blue' size='2'>Note:  Always run cells with blue headers before running any other processes.

# <font color=blue>Import modules</font>

In [ ]:
import os, sys
import arcgis
from arcgis.gis import GIS
import json
from json import loads
import csv
from datetime import date
from datetime import datetime
from colorama import Fore, Back, Style
print(Fore.BLUE+"Modules Imported "+Style.RESET_ALL+str(datetime.now().strftime("%m/%d/%Y %H:%M "))+(time.localtime().tm_zone))

# <font color=blue>Project variables</font>

In [ ]:
### INPUT CELL ###
# Target portal URL
portalURL = 'https://epa.maps.arcgis.com/'

# Specify a project name:
projectName = 'ProjectName' #used in output folder names and paths created, suggested to use no spaces.

# Specify an output directory for csv file in "outputFolder" variable.  Concatenates with other variables:
folder_datestamp = str(datetime.now().strftime("_%Y%m%d")) #Do not edit
folder_timestamp = str(datetime.now().strftime("%H%M")) #Do not edit
outputFolder=str('E:\\ContractFolder\\ProjectSubfolder\\ScriptsSubfolder\\'+projectName+folder_datestamp+'\\'+folder_timestamp+'\\') #format example: str('C:\\PythonTemp\\')

# Provide keywords to search Geoplatform item titles and tags:
tagKeywordList = ['Tag keywords 1','Tag keywords 2','Tag keywords 3'] #used to search item tags and names
print(Fore.BLUE+"Search keywords: "+Style.RESET_ALL+str(tagKeywordList))

# Provide list(s) of Geoplatform user names relevant to the project (for searching user content):
usersR9 = ['User1_EPAEXT','User2_EPAEXT','User3_EPAEXT','User4_EPAEXT', \
           'User5_EPAEXT',]

usersSTART = ['User6_EPAEXT','User7_EPAEXT','User8_EPAEXT', \
              'User9_EPAEXT','User10_EPAEXT','User11_EPAEXT','User12_EPAEXT', \
              'User13_EPAEXT','User14_EPAEXT']

usersR9EPA = ['User1_EPA','User2_EPA']
            
usersR9TC = ['User15_EPAEXT','User16_EPAEXT','User17_EPAEXT','User18_EPAEXT', \
             'User19_EPAEXT','User20_EPAEXT']

usersOther = ['User21_EPAEXT']

# Choose userNames to assebmle from lists above: 1 = R9 START GIS, 2 = All START GIS, 3 = R9 EPA, 4 = R9 Tech Center, 5 = Other
usersOptions = (1,2,3,4,5) # enter only the numbers that apply from list above.

### INPUT SECTION END###

In [ ]:
#Function to assemble list of userNames from numeric options set in usersOptions variable:
def appendUsernamesList():
    usersDictionary = {1 : usersR9, 2 : usersSTART, 3 : usersR9EPA, 4 : usersR9TC, 5 : usersOther}
    userNamesCache =[]
    for usersOption in usersOptions:
        appendNames = userNamesCache+(usersDictionary[usersOption])
        userNamesCache = appendNames
    return userNamesCache
userNames = appendUsernamesList()
print(Fore.BLUE+"Username list: "+Style.RESET_ALL+str(userNames))

#Checks if the required directory exists, and creates it if not: 
print(Fore.BLUE+"Output folder: "+Style.RESET_ALL+outputFolder)
def createOutputDirectories():
    if os.path.isdir(outputFolder) is True:
        print(Fore.RED+"Directory exists: "+Style.RESET_ALL+str(outputFolder))
    elif os.path.isdir(outputFolder) is False:
        os.makedirs(outputFolder)
        print(Fore.RED+"Directory created: "+Style.RESET_ALL+str(outputFolder))
    else:
        print(Fore.RED+"Problem with outputFolder variable."+Style.RESET_ALL)
createOutputDirectories()

# <font color=blue>Connect to Portal</font>
- <font color=red>Important:</font>  Connection method in this script requires you to have ArcGIS Pro logged into the correct ArcGIS Online Organization target (U.S. EPA Geoplatform).

In [ ]:
### ArcGIS portal url and login ###
portal = GIS('pro')
token = portal._con.token
print(Fore.BLUE+"Logged in as: "+Style.RESET_ALL+str(portal.properties.user.username))

# 1) List Web Map and Web Scene Layer Dependencies
- Generates a list of all LAYERS in each Geoplatform WEB MAP based on specified tags and usernames.
- Normally this list is used to determine the list of ER Cloud service layers that will need to be re-published up to the Geoplatform **ONLY IF** EPA says that WEB MAP will need to remain operational after service layers on the ER Cloud have been archived and removed.
- Examples: keeping a web map operational as an example for future similar projects, or keeping a public app open after the project.

In [ ]:
# Create csv output file for standard WEB MAP and WEB SCENE dependencies decommissioning list:
dateForFilename=date.today()
csv1 = open(outputFolder+"1a_"+projectName+"_DependenciesList_WebMap_"+str(dateForFilename)+".csv", "w")
csv1.write("Web Map, Map Owner, Web Map Item ID, Web Map Sharing, Layer Order, Layer Title in Web Map, Layer Item Name, Layer Item ID, Layer Type, Layer Owner, Layer Sharing, Data Repository, Map Service URL, GeoPlatform URL" + '\n')
print(Fore.BLUE+"Output standard CSV: "+Style.RESET_ALL+outputFolder+"1a_"+projectName+"_DependenciesList_WebMap_"+str(dateForFilename)+".csv")

# (Neo4J use only) Create target csv output file for Neo4J (feature services sub-list version):
csv2 = open(outputFolder+"1b_"+projectName+"_DependenciesList_WebMap_"+str(dateForFilename)+".csv", "w")
csv2.write("Web_Map, Map_Owner, Web_Map Item ID, Web_Map Sharing, Layer_Order, Layer_Title_in_Web_Map, Layer_Item_Name, Layer_Item_ID, Layer_Type, Layer_Owner, Layer_Sharing, Data_Repository, Map_Service URL, GeoPlatform_URL" + '\n')
print(Fore.BLUE+"Output Neo4J services CSV: "+Style.RESET_ALL+outputFolder+"1b_"+projectName+"Neo4Jservices_DependenciesList_WebMap"+str(dateForFilename)+".csv")

# (Neo4J use only) Create target csv output file for Neo4J (web maps sub-list version):
csv3 = open(outputFolder+"1c_"+projectName+"_DependenciesList_WebMap_"+str(dateForFilename)+".csv", "w")
csv3.write("Web_Map, Map_Owner, Web_Map Item ID, Web_Map Sharing, Layer_Order, Layer_Title_in_Web_Map, Layer_Item_Name, Layer_Item_ID, Layer_Type, Layer_Owner, Layer_Sharing, Data_Repository, Map_Service URL, GeoPlatform_URL" + '\n')
print(Fore.BLUE+"Output Neo4J web maps CSV: "+Style.RESET_ALL+outputFolder+"1c_"+projectName+"Neo4Jservices_DependenciesList_WebMap"+str(dateForFilename)+".csv")

In [ ]:
#Trying several methods of pulling URLs for web maps from AGOL, which is buggy. URL's may be missing in output csv file:
def get_source_url(item, lyrDict):
    
    sourceURL=''
    try:
        sourceURL=item.sourceUrl
        #print ("item.sourceUrl")
        #print (sourceURL)
        return sourceURL
    except:
        try:
            sourceURL=lyrDict['templateUrl']
            #print (r"lyrDict['templateUrl']")
            #print (sourceURL)
            return sourceURL
        except:
            try:
                sourceURL=lyrDict['ArcGISMapServiceLayer']
                #print (r"lyrDict['ArcGISMapServiceLayer']")
                #print (sourceURL)
                return sourceURL
            except:
                try:
                    sourceURL=lyrDict['url']
                    #print (r"lyrDict['url']")
                    #print (sourceURL)
                    return sourceURL
                except:
                    sourceURL="no url found"
                    #print(sourceURL)
                    return sourceURL
                
def get_map_item_dependencies(source_portal, item, folder, user):
    print(Fore.BLUE+"Found "+item.type+":  "+Style.RESET_ALL+Back.BLUE+Fore.YELLOW+item.title+Style.RESET_ALL+", ID: "+item.id)
    # Using JSON information here to get info about 'Web Map' items; python item queries are buggy on AGOL for web maps:
    if item.type == 'Web Map' or item.type == 'Web Scene':
        # Get the item's data in JSON format
        item_data = item.get_data(False)
        data = dict(loads(item_data))
        cnt=0 #resetting for total count below.
        for layer_count in data['operationalLayers']: #first for loop just counts total layers.
            cnt+=1 #counting total number of layers
        print (Fore.BLUE+"Layer Count=",str(cnt)+'.  Searching for layers...'+Style.RESET_ALL)
        cnt=0 #resetting after total layer count print statement above, prepare to count each layer.
        
        for layer_item in data['operationalLayers']: # Operational Layers are the main dependent items:
            cnt+=1 #count each layer in operationalLayers loop.
            try:
                # test if it has "itemId", doesn't write anything to csv file yet:
                layer_itemInfo = source_portal.content.get(layer_item['itemId']) if 'itemId' in layer_item else ''
                sdItem=source_portal.content.get(layer_item['itemId'])
                layerItemID=sdItem.id
                print(Fore.RED+"  Layer: "+str(cnt)+"  "+Style.RESET_ALL+" {}, ID: {}".format(sdItem.title, sdItem.id))
                                                
            except Exception as e: #exception 1
                print(Fore.RED+"  Layer: "+str(cnt)+"   "+Style.RESET_ALL+Fore.RED+"Could not get item from portal. Error: "+str(e)+Style.RESET_ALL)
                try:
                    # if no itemId or id, write error with layer_item["title"] from dictionary output from web map:
                    print("         Title: "+Fore.YELLOW+layer_item["title"]+Style.RESET_ALL)
                    print(Fore.YELLOW+"         Printing layer URL for reference..."+Style.RESET_ALL)
                    print(str(layer_item["layers"][0]["url"]))
                    print(Fore.YELLOW+"         Trying alternate method to get itemId key in layer dictionary...")
                                        
                    joinInput=((str(item.title).replace(',',"")), #Web Map
                               user, #Map Owner
                               item.id, #Web Map Item ID
                               (str(item.shared_with).replace(','," & ")), #Web Map Sharing
                               str(cnt), #Layer Order
                               (str(layer_item['title']).replace(',',"")), #Layer Title in Web Map
                               '"'+layer_item['title']+'"', #Layer Item Name 
                               "none", #Layer Item ID
                               layer_item['layerType'], #Layer Type
                               "", #Layer Owner
                               "", #layer_item.shared_with, #Layer Sharing
                               "", #Data Repository
                               "", #Map Service URL
                               layer_item["layers"][0]['url']) #Geoplatform URL
                    csv1.write(", ".join(joinInput) + '\n')
                    print(Fore.GREEN+"         ...alternate joinInput write succeeded."+Style.RESET_ALL)
                    #write to separate csv's for Neo4J output as well:
                    if item.type == 'Web Map':
                        csv3.write(", ".join(joinInput) + '\n') #writing to web map Neo4J csv.
                    else:
                        csv2.write(", ".join(joinInput) + '\n') #writing to services Neo4J csv.
                    
                except Exception as e: #exception 2                  
                    # if other error print generic error
                    print(Fore.RED+"         Error with alternate joinInput method.  Error:  "+str(e)+Style.RESET_ALL)
                    print("         "+Back.RED+"Writing layer title to csv without layer item ID, url, or owner name."+Style.RESET_ALL)
                    joinInput=((str(item.title).replace(',',"")), #Web Map
                               user, #Map Owner
                               item.id, #Web Map Item ID
                               (str(item.shared_with).replace(','," & ")), #Web Map Sharing
                               str(cnt), #Layer Order
                               (str(layer_item['title']).replace(',',"")), #Layer Title in Web Map
                               str(layer_item['id']), #Layer Item Name
                               "no item id", #Layer Item ID
                               layer_item['layerType'], #Layer Type
                               "unknown owner", #Layer Owner
                               "", #layer_item.shared_with, #Layer Sharing
                               "unknown", #Data Repository
                               "none", #Map Service URL
                               "no url") #Geoplatform URL
                    csv1.write(", ".join(joinInput) + '\n')
                    #write to separate csv's for Neo4J output as well:
                    if item.type == 'Web Map':
                        csv3.write(", ".join(joinInput) + '\n') #writing to web map Neo4J csv.
                    else:
                        csv2.write(", ".join(joinInput) + '\n') #writing to services Neo4J csv.                    
                continue

            #format URL
            sourceURL=get_source_url(sdItem, layer_item)
            dataRepo=""
            urlClean1=sourceURL.replace("https://", "")
            urlList=urlClean1.split(r"/")
            dataRepo=urlList[0]
            
            try:
                # if found item print FULL details
                joinInput=((str(item.title).replace(',',"")), #Web Map
                               user, #Map Owner
                               item.id, #Web Map Item ID
                               (str(item.shared_with).replace(','," & ")), #Web Map Sharing
                               str(cnt), #Layer Order
                               (str(layer_item['title']).replace(',',"")), #Layer Title in Web Map
                               (str(sdItem.title).replace(',',"")), #Layer Item Name
                               layerItemID, #Layer Item ID
                               layer_item['layerType'], #Layer Type
                               layer_itemInfo.owner if layer_itemInfo and 'owner' in layer_itemInfo else 'Unknown Owner', #Layer Owner
                               (str(sdItem.shared_with).replace(','," & ")), #Layer Sharing
                               dataRepo, #Data Repoository    
                               sourceURL, #Map Service URL
                               sdItem.url) #Geoplatform URL     
                #print(", ".join(joinInput) + '\n') #for troubleshooting joinInput.
                csv1.write(", ".join(joinInput) + '\n')
                #write to separate csv's for Neo4J output as well:
                if item.type == 'Web Map':
                    csv3.write(", ".join(joinInput) + '\n') #writing to web map Neo4J csv.
                else:
                    csv2.write(", ".join(joinInput) + '\n') #writing to services Neo4J csv.
                
            except Exception as e:
                print(Fore.RED+"         Primary joinInput method failed to write, no alternate method available!"+Style.RESET_ALL)
                print("         Error: "+Back.RED+str(e)+Style.RESET_ALL)
                print(Fore.RED+"         Not written to csv:"+Style.RESET_ALL)
                print("         Item ID: "+Fore.RED+layerItemID+Style.RESET_ALL)
                print("         URL: "+Fore.RED+sourceURL+Style.RESET_ALL)
                continue
            
        # Get TABLES
        try: 
            y=data['tables']
        except:
            print("     No tables")
            
        try:
            tcnt=0 #preparing to count tables
            for tbl_item in data['tables']:
                tcnt+=1
                print(Fore.CYAN+"  Table "+str(tcnt)+":  "+Style.RESET_ALL+tbl_item['title']+", ID: "+tbl_item['itemId'])
                print("         URL: "+tbl_item['url'])
                #write to csv
                joinInput=(item.title, #Web Map
                               user, #Map Owner
                               item.id, #Web Map Item ID
                               (str(item.shared_with).replace(','," & ")), #Web Map Sharing
                               'Table', #Layer Order
                               '"'+tbl_item["title"]+'"', #Layer Title in Web Map
                               tbl_item["id"], #Layer Item Name
                               tbl_item['itemId'], #Layer Item ID
                               "Table", #Layer Type
                               (portal.content.get(tbl_item['itemId'])).owner, #Layer Owner
                               (str((portal.content.get(tbl_item['itemId'])).shared_with).replace(','," & ")), #Layer Sharing
                               '', #Data Repository
                               '', #Map Service URL
                               tbl_item['url']) # Geoplatform URL
                #print(", ".join(joinInput) + '\n')
                csv1.write(", ".join(joinInput) + '\n')
                #write to separate csv's for Neo4J output as well:
                if item.type == 'Web Map':
                    csv3.write(", ".join(joinInput) + '\n') #writing to web map Neo4J csv.
                else:
                    csv2.write(", ".join(joinInput) + '\n') #writing to services Neo4J csv.
        except Exception as e:
            print ("     Error with table joinInput:  "+Back.RED+str(e)+Style.RESET_ALL)
    
def loopKeywords(item): # Search Geoplatform item titles and tags for key words.
    found=''
    for tagKeyword in tagKeywordList:
        if tagKeyword in item.title or tagKeyword in item.tags:
            found=1
            break  
    return found
    
def searchGP_UsersKeywords():
        
    for userName in userNames:
        try:
            print("Searching content for user:",Back.GREEN+userName+Style.RESET_ALL,"...")
            user = portal.users.get(userName)
            user_content = user.items(max_items=5000)

            # Get item ids from root folder first
            for item in user_content:

                if (item.type == 'Web Map' or item.type == 'Web Scene'):
                    found2=loopKeywords(item)
                    if found2==1:
                        get_map_item_dependencies(portal, item, 'Root Folder', userName)

            # Get item ids from each of the folders next
            folders = user.folders
            for folder in folders:
                folder_items = user.items(folder=folder['title'],max_items=5000)
                for item in folder_items:
                    if (item.type == 'Web Map' or item.type == 'Web Scene'):
                        found2=loopKeywords(item)
                        if found2==1:
                            get_map_item_dependencies(portal, item, folder['title'], userName)
        except:
            print(Fore.Red+"Could not access content for user:  "+userName+Style.RESET_ALL)
                        
searchGP_UsersKeywords()
csv1.close()
csv2.close()
csv3.close()
print(Fore.GREEN+"END"+Style.RESET_ALL)

# 2) List Dashboard Dependencies
- Generates a list of all dependencies in each Geoplatform DASHBOARD based on specified tags and usernames.
- Normally this list is used to determine the list of ER Cloud service layers that will need to be re-published up to the Geoplatform **ONLY IF** EPA says that DASHBOARD will need to remain operational after service layers on the ER Cloud have been archived and removed.
- Examples: keeping a dashboard operational as an example for future similar projects, or keeping a public app open after the project.

In [ ]:
# Create csv output file for standard DASHBOARD dependencies decommissioning list:
dateForFilename=date.today()
csv4 = open(outputFolder+"2a_"+projectName+"_DependenciesList_Dashboard_"+str(dateForFilename)+".csv", "w")
csv4.write("Dashboard, Dashboard Owner, Dashboard Item ID, Dashboard Sharing, Widget Order, Widget Source Layer#, Layer Title, Layer Item ID, Layer Type, Layer Owner, Layer URL, Layer Sharing" + '\n')
print(Fore.BLUE+"Output standard CSV: "+Style.RESET_ALL+outputFolder+"2a_"+projectName+"_DependenciesList_Dashboard_"+str(dateForFilename)+".csv")

# (Neo4J use only) Create csv output file for Neo4J (feature services sub-list version):
csv5 = open(outputFolder+"2b_"+projectName+"_Neo4Jservices_DependenciesList_Dashboard"+str(dateForFilename)+".csv", "w")
csv5.write("Dashboard, Dashboard_Owner, Dashboard_Item_ID, Dashboard_Sharing, Widget_Order, Widget_Source_Layer#, Layer_Title, Layer_Item_ID, Layer_Type, Layer_Owner, Layer_URL, Layer_Sharing" + '\n')
print(Fore.BLUE+"Output Neo4J services CSV: "+Style.RESET_ALL+outputFolder+"2b_"+projectName+"_Neo4Jservices_DependenciesList_Dashboard_"+str(dateForFilename)+".csv")

# (Neo4J use only) Create csv output file for Neo4J (web maps sub-list version):
csv6 = open(outputFolder+"2c_"+projectName+"_Neo4Jwebmaps_DependenciesList_Dashboard"+str(dateForFilename)+".csv", "w")
csv6.write("Dashboard, Dashboard_Owner, Dashboard_Item_ID, Dashboard_Sharing, Widget_Order, Widget_Source_Layer#, Layer_Title, Layer_Item_ID, Layer_Type, Layer_Owner, Layer_URL, Layer_Sharing" + '\n')
print(Fore.BLUE+"Output Neo4J web maps CSV: "+Style.RESET_ALL+outputFolder+"2c_"+projectName+"_Neo4Jwebmaps_DependenciesList_Dashboard_"+str(dateForFilename)+".csv")

In [ ]:
def get_db_dependencies(sourcePortal, item, folder, user):
    print(Fore.BLUE+"Found "+item.type+":  "+Style.RESET_ALL+Back.BLUE+Fore.YELLOW+item.title+Style.RESET_ALL+", ID: "+item.id)
    if item.type == 'Dashboard':
        jsonWidgets = (item.get_data()).get("desktopView")["widgets"]
        cnt=0 #preparing to count widgets
        for widget in jsonWidgets:
            cnt+=1 #counting widgets for output
            widgetSuccess=0 #preparing to track which csv write process to use (joinInput variable below this section)
            try: #try is checking if need to dig into "datasets" and "dataSource" keys to get to itemID key.
                widgetSource = widget.get("datasets")[0]["dataSource"] #looking into widget's keys to see if dataSource present.
                widgetItem = sourcePortal.content.get(widgetSource["itemId"]) #get widget dataSource item from portal.
                widgetType = str(widgetSource["layerId"]) #if dataSource is a layer, making type = layerId for output.
                widgetSuccess+=1 #using to determine which joinInput method to use.
                print(Fore.RED+"  Widget: "+str(cnt)+"  "+Style.RESET_ALL+" {}, ID: {}, Layer: {}".format(widgetItem.title, widgetItem.id, str(widgetSource["layerId"])))
            except: #any widget types in the except should have no itemId key, or the key is directly callable without digging into sub-arrays.
                try:
                    #print("else text: "+str(widget)) #for troubleshooting
                    widgetItem = sourcePortal.content.get(widget["itemId"]) #if widget does not have key dataSource, using itemId key.
                    widgetType = str(widget["type"])
                    widgetSuccess+=1 #using to determine which joinInput method to use.
                    print(Fore.RED+"  Widget: "+str(cnt)+"  "+Style.RESET_ALL+" {}, ID: {}, Layer: {}".format(widgetItem.title, widgetItem.id, str(widget["type"])))
                except:
                    widgetType = widget["type"]+" - no data source"
                    print(Fore.RED+"  Widget: "+str(cnt)+"  "+Style.RESET_ALL+widgetType)
            
            # Write to CSV
            if widgetSuccess==1: #use this joinInput if the widget has a valid data source itemId in Geoplatform.
                joinInput=(item.title, # Dash Title
                            item.owner, #Dash Owner
                            item.id, #Dash Item ID
                            (str(item.shared_with).replace(','," & ")), #Dash Sharing
                            str(cnt), #Widget Order
                            widgetType, #gather a type 3 possiblilites in try above.
                            widgetItem.title, #Layer Title
                            widgetItem.id, #Layer Item ID
                            widgetItem.type, #Layer Type
                            widgetItem.owner, #Layer Owner  
                            widgetItem.url, #Layer URL
                            (str(widgetItem.shared_with).replace(','," & "))) #Layer Sharing
            else: #use this joinInput if the widget has no data source (text or legend widget types).
                joinInput=(item.title, # Dash Title
                            item.owner, #Dash Owner
                            item.id, #Dash Item ID
                            (str(item.shared_with).replace(','," & ")), #Dash Sharing
                            str(cnt), #Widget Order
                            widgetType, #gather a type 3 possiblilites in try above.
                            str("None"), #Layer Title
                            str("None"), #Layer Item ID
                            str("None"), #Layer Type
                            str("None"), #Layer Owner  
                            str("None"), #Layer URL
                            str("None")) #Layer Sharing
            #print(", ".join(joinInput) + '\n') #placeholder for troubleshooting.
            csv4.write(", ".join(joinInput) + '\n')
            #write to separate csv's for Neo4J output as well:
            if widgetItem.type == 'Web Map':
                csv6.write(", ".join(joinInput) + '\n') #writing to web map Neo4J csv.
            else:
                csv5.write(", ".join(joinInput) + '\n') #writing to services Neo4J csv.      

def loopKeywords(item): # Search Geoplatform item titles and tags for key words.
    found=''
    for tagKeyword in tagKeywordList:
        if tagKeyword in item.title or tagKeyword in item.tags:
            found=1
            break  
    return found
    
def searchGP_UsersKeywords():
    try:
        for userName in userNames:
            print("Searching content for user:",Back.GREEN+userName+Style.RESET_ALL,"...")
            user = portal.users.get(userName)
            user_content = user.items(max_items=5000)

            # Get item ids from root folder first
            for item in user_content:
                if item.type == 'Dashboard':
                    found2=loopKeywords(item)
                    if found2==1:
                        get_db_dependencies(portal, item, 'Root Folder', userName)

            # Get item ids from each of the folders next
            folders = user.folders
            for folder in folders:
                folder_items = user.items(folder=folder['title'],max_items=5000)
                for item in folder_items:
                    if item.type == 'Dashboard':
                        found2=loopKeywords(item)
                        if found2==1:
                            get_db_dependencies(portal, item, folder['title'], userName)
    except:
            print(Fore.Red+"Could not access content for user:  "+userName+Style.RESET_ALL)

searchGP_UsersKeywords()
csv4.close()
csv5.close()
csv6.close()
print(Fore.GREEN+"END"+Style.RESET_ALL)

# 3) List Experience Builder Dependencies
- Generates a list of all dependencies in each Geoplatform EXPERIENCE BUILDER based on specified tags and usernames.
- Normally this list is used to determine the list of ER Cloud service layers that will need to be re-published up to the Geoplatform **ONLY IF** EPA says that EXPERIENCE BUILDER will need to remain operational after service layers on the ER Cloud have been archived and removed.
- Examples: keeping an experience builder operational as an example for future similar projects, or keeping a public app open after the project.

In [ ]:
# Create csv output file for standard EXPERIENCE BUILDERS dependencies decommissioning list:
dateForFilename=date.today()
csv7 = open(outputFolder+"3a_"+projectName+"_DependenciesList_ExBuilder_"+str(dateForFilename)+".csv", "w")
csv7.write("Experience Builder, ExBld Owner, ExBld Item ID, ExBld Sharing, DataSource Order, Data Title, Data Item ID, Data Type, Data Owner, Data URL, Data Sharing" + '\n')
print(Fore.BLUE+"Output standard CSV: "+Style.RESET_ALL+outputFolder+"3a_"+projectName+"_DependenciesList_ExBuilder_"+str(dateForFilename)+".csv")

# (Neo4J use only) Create csv output file for Neo4J (feature services sub-list version):
csv8 = open(outputFolder+"3b_"+projectName+"_Neo4Jservices_DependenciesList_Dashboard"+str(dateForFilename)+".csv", "w")
csv8.write("Experience_Builder, ExBld_Owner, ExBld_Item_ID, ExBld_Sharing, DataSource_Order, Data_Title, Data_Item_ID, Data_Type, Data_Owner, Data_URL, Data_Sharing" + '\n')
print(Fore.BLUE+"Output Neo4J services CSV: "+Style.RESET_ALL+outputFolder+"3b_"+projectName+"_Neo4Jservices_DependenciesList_ExBuilder_"+str(dateForFilename)+".csv")

# (Neo4J use only) Create csv output file for Neo4J (web maps sub-list version):
csv9 = open(outputFolder+"3c_"+projectName+"_Neo4Jwebmaps_DependenciesList_Dashboard"+str(dateForFilename)+".csv", "w")
csv9.write("Experience_Builder, ExBld_Owner, ExBld_Item_ID, ExBld_Sharing, DataSource_Order, Data_Title, Data_Item_ID, Data_Type, Data_Owner, Data_URL, Data_Sharing" + '\n')
print(Fore.BLUE+"Output Neo4J web maps CSV: "+Style.RESET_ALL+outputFolder+"3c_"+projectName+"_Neo4Jwebmaps_DependenciesList_ExBuilder_"+str(dateForFilename)+".csv")

In [ ]:
def get_exbld_dependencies(item, portal):
    itemData = item.get_data()
    print(Fore.BLUE+"Found "+item.type+":  "+Style.RESET_ALL+Back.BLUE+Fore.YELLOW+item.title+Style.RESET_ALL+", ID: "+item.id)
    try:
        dataSources = itemData.get("dataSources")
        #print(dataSources) #for troubleshooting dataSources dictionary.
        dataSourceItemIDs = dataSources.keys()
        #print("dataSourceItemIDs: "+str(dataSourceItemIDs)) #for troubleshooting list of item IDs from JSON dataSources.
        cnt=0 #preparing to count dataSources
        for dataSourceItemID in dataSourceItemIDs:
            cnt+=1 #counting dataSources keys for output
            dsItem = portal.content.get(dataSourceItemID)
            print(Fore.RED+"  DataSource: "+str(cnt)+Style.RESET_ALL+" "+dsItem.title+", ID: "+dsItem.id+", Type: "+dsItem.type)
            try:
                joinInput=(item.title, # Dash Title
                        item.owner, #ExBld Owner
                        item.id, #ExBld Item ID
                        (str(item.shared_with).replace(','," & ")), #ExBld Sharing
                        str(cnt), #dataSource key order
                        dsItem.title, #Layer Title
                        dsItem.id, #Layer Item ID
                        dsItem.type, #Layer Type
                        dsItem.owner, #Layer Owner  
                        dsItem.url, #Layer URL
                        (str(dsItem.shared_with).replace(','," & "))) #Layer Sharing
                #print(", ".join(joinInput) + '\n') #placeholder for troubleshooting.
                csv7.write(", ".join(joinInput) + '\n') #writing to decommissioning csv.
                #write to separate csv's for Neo4J output as well:
                if dsItem.type == 'Web Map':
                    csv9.write(", ".join(joinInput) + '\n') #writing to web map Neo4J csv.
                else:
                    csv8.write(", ".join(joinInput) + '\n') #writing to services Neo4J csv.

            except:
                print(Fore.RED+"Writing to csv file failed, likely due to joinInput variable."+Style.RESET_ALL)

    except:
        print(Fore.RED+"  Failed to get valid Item ID's from dataSources dictionary key.  Check formatting of dataSources key in JSON."+Style.RESET_ALL)

            
def loopKeywords(item): # Search Geoplatform item titles and tags for key words.
    found=''
    for tagKeyword in tagKeywordList:
        if tagKeyword in item.title or tagKeyword in item.tags:
            found=1
            break  
    return found
    
def searchGP_UsersKeywords():    
    try:   
        for userName in userNames:
            print("Searching content for user:",Back.GREEN+userName+Style.RESET_ALL,"...")
            user = portal.users.get(userName)
            user_content = user.items(max_items=5000)

            # Get item ids from root folder first
            for item in user_content:  
                if (item.type == 'Web Experience'):
                    found2=loopKeywords(item)
                    if found2==1:
                        get_exbld_dependencies(item, portal)

            # Get item ids from each of the folders next
            folders = user.folders
            for folder in folders:
                folder_items = user.items(folder=folder['title'],max_items=5000)
                for item in folder_items:
                    if (item.type == 'Web Experience'):
                        found2=loopKeywords(item)
                        if found2==1:
                            get_exbld_dependencies(item, portal)
    except:
            print(Fore.Red+"Could not access content for user:  "+userName+Style.RESET_ALL)

searchGP_UsersKeywords()
csv7.close()
csv8.close()
csv9.close()
print(Fore.GREEN+"END"+Style.RESET_ALL)

# <font color=magenta>Do not run - Development section for experience builder widgets</font>

In [ ]:
item = portal.content.get("4649bd8d472c4375a95544e0b08a314a") #remove
#itemData = item.get_data()
print(Fore.BLUE+"Found "+item.type+":  "+Style.RESET_ALL+Back.BLUE+Fore.YELLOW+item.title+Style.RESET_ALL+", ID: "+item.id)
try:
    jsonWidgets = (item.get_data())["widgets"]
    cnt=0 #preparing to count widgets
    for widget in jsonWidgets:
        widgetData = jsonWidgets.get(str(widget))
        cnt+=1 #counting widgets for output
        widgetSuccess=0 #preparing to track which csv write process to use (joinInput variable below this section)
        try: #try is checking if need to dig into "datasets" and "dataSource" keys to get to itemID key.
            widgetLabel = widgetData.get("label") #looking into widget's keys to print widget label.
            try: #try getting widget config (config key only exists in widgets with data sources)
                widgetConfig = widgetData.get("config")
                widgetURL = widgetConfig.get("expression").strip("<//p>")
            except:
                print(Fore.RED+"  Widget "+str(cnt)+": "+Style.RESET_ALL+Fore.YELLOW+"No data source in JSON for this widget type. "+Style.RESET_ALL+"Widget label = "+widgetLabel)
                continue
            if widgetConfig["embedType"] == "url":
                widgetSuccess+=1
                widgetItemID = widgetURL[-32:]
                widgetItem = portal.content.get(str(widgetItemID))
                print(Fore.RED+"  Widget "+str(cnt)+": "+Style.RESET_ALL+widgetItem.title+", ID: "+widgetItem.id)
            else:
                continue
        except:
            print("Failure in for loop, first try statement. Check code against item JSON.")


except:
    print(Fore.RED+"  Failed to get valid Item ID's from dataSources dictionary key.  Check formatting of dataSources key in JSON."+Style.RESET_ALL)

